In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py
# !wget https://huggingface.co/datasets/mesolitica/instructions-dataset/resolve/main/shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [2]:
# !split -l 213000 -d --additional-suffix=.splitted shuf-combine-malay-no-alignment-multitasks-v5.jsonl shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [3]:
!ls shuf-combine-malay-no-alignment-multitasks-v5.jsonl*

shuf-combine-malay-no-alignment-multitasks-v5.jsonl
shuf-combine-malay-no-alignment-multitasks-v5.jsonl00.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl01.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl02.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl03.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl04.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl05.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl06.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl07.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl08.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl09.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl10.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl11.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl12.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl13.splitted
shuf-combine-malay-no-alignment-multitasks-v5.jsonl14.splitted
shu

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b-it')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
tokenizer.padding_side = "right"

In [5]:
def generate_and_tokenize_prompt(row):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    return {'text': prompt}

In [6]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
from glob import glob
import os
import json

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

columns = {
    'input_ids': 'uint16',
}
hashes = 'sha1', 'xxh64'

In [7]:
!mkdir tokenized_gemma

In [8]:
def loop(files, block_size = 16384):
    files, index = files
    out_root = f'tokenized_gemma/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    temp = []
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for f in files:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    row = json.loads(l)
                    element = generate_and_tokenize_prompt(row)
                    outputs = tokenizer(element['text'])
                    temp.extend(outputs['input_ids'])
                    done = False
                    while len(temp) >= block_size:
                        block = temp[:block_size]
                        temp = temp[block_size:]
                        if len(block) == block_size:
                            out.write({
                                'input_ids': np.array(block).astype(np.uint16)
                            })

In [9]:
files = sorted(glob('shuf-combine-malay-no-alignment-multitasks-v5.jsonl*.splitted'), key = lambda x: int(x.split('jsonl')[-1].split('.')[0]))
files

['shuf-combine-malay-no-alignment-multitasks-v5.jsonl00.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl01.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl02.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl03.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl04.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl05.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl06.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl07.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl08.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl09.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl10.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl11.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl12.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl13.splitted',
 'shuf-combine-malay-no-alignment-multitasks-v5.jsonl14.splitt

In [10]:
import mp
mp.multiprocessing(files, loop, cores = min(len(files), 30), returned = False)

26390it [01:04, 476.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

196786it [06:50, 479.85it/s]
213000it [07:18, 485.86it/s]
213000it [07:18, 485.81it/s]
213000it [07:18, 485.80it/s]
213000it [07:18, 485.75it/s]
213000it [07:20, 483.94it/s]
213000it [07:21, 482.57it/s]
213000it [07:22, 481.69it/s]
213000it [07:23, 480.77it/s]
213000it [07:23, 480.60it/s]
213000it [07:25, 478.18it/s]
213000it [07:25, 478.08it/s]
213000it [07:25, 477.65it/s]
171855it [07:26, 333.66it/s]
213000it [07:26, 476.73it/s]
213000it [07:26, 476.72it/s]
213000it [07:27, 475.93it/s]
213000it [07:39, 463.70it/s]
213000it [07:51, 452.06it/s]
213000it [09:02, 392.29it/s]


In [13]:
folders = sorted(glob('tokenized_gemma/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['tokenized_gemma/tokenized-0',
 'tokenized_gemma/tokenized-1',
 'tokenized_gemma/tokenized-2',
 'tokenized_gemma/tokenized-3',
 'tokenized_gemma/tokenized-4',
 'tokenized_gemma/tokenized-5',
 'tokenized_gemma/tokenized-6',
 'tokenized_gemma/tokenized-7',
 'tokenized_gemma/tokenized-8',
 'tokenized_gemma/tokenized-9',
 'tokenized_gemma/tokenized-10',
 'tokenized_gemma/tokenized-11',
 'tokenized_gemma/tokenized-12',
 'tokenized_gemma/tokenized-13',
 'tokenized_gemma/tokenized-14',
 'tokenized_gemma/tokenized-15',
 'tokenized_gemma/tokenized-16',
 'tokenized_gemma/tokenized-17',
 'tokenized_gemma/tokenized-18',
 'tokenized_gemma/tokenized-19']

In [14]:
!rm -rf packing-gemma

In [15]:
with MDSWriter(out='packing-gemma', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 8714/8714 [00:00<00:00, 10621.90it/s]


In [16]:
dataset = LocalDataset('packing-gemma')

In [17]:
len(dataset) * 16384

3069935616

In [18]:
dataset[-2]

{'input_ids': array([38404, 38661, 11599, ..., 38666,  6707, 21795], dtype=uint16)}